# Explaining unsatisfiability

In this tutorial, we are using tools [cpmpy](https://cpmpy.readthedocs.io/en/latest/) to explain unsatifiable problems.

More precisely we are using our wrapper for cpmpy `CpmpySolver` (and derived classes for specific problems) which makes use of `cpmpy.model.Model` but also of the explaining tools from `cpmpy.tools.explain`.

To illustrate it, we will use the [coloring problem](https://en.wikipedia.org/wiki/Graph_coloring): it consists in coloring vertices of a graph with the minimal number of colors, such that 2 adjacent vertices do not have the same color.

<img src="https://upload.wikimedia.org/wikipedia/commons/9/90/Petersen_graph_3-coloring.svg" alt="Petersen graph 3-coloring.svg"  width="280">

## Prerequisites

Concerning the python kernel to use for this notebook:
- If running locally, be sure to use an environment with discrete-optimization;
- If running on colab, the next cell does it for you;
- If running on binder, the environment should be ready.


In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import sys  # noqa: avoid having this import removed by pycln

    !{sys.executable} -m pip install -U pip

    # uninstall google protobuf conflicting with ray and sb3
    ! pip uninstall -y protobuf

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install git+https://github.com/airbus/discrete-optimization@master#egg=discrete-optimization

### Imports

In [ ]:
from discrete_optimization.coloring.parser import get_data_available, parse_file
from discrete_optimization.coloring.plot import (
    plot_coloring_problem,
    plot_coloring_solution,
)
from discrete_optimization.coloring.solvers.cpmpy import CpmpyColoringSolver
from discrete_optimization.datasets import fetch_data_from_coursera

### Download datasets

If not yet available, we import the datasets from [coursera](https://github.com/discreteoptimization/assignment).

In [ ]:
needed_datasets = ["gc_20_1"]
download_needed = False
try:
    files_available_paths = get_data_available()
    for dataset in needed_datasets:
        if len([f for f in files_available_paths if dataset in f]) == 0:
            download_needed = True
            break
except:
    download_needed = True

if download_needed:
    fetch_data_from_coursera()

### Instantiate the color problem

We choose here a small problem to have solvers running fast but feel free to choose a more complex one.

In [ ]:
instance = "gc_20_1"
filepath = [f for f in get_data_available() if instance in f][0]
color_problem = parse_file(filepath)

Let us have a look to the underlying graph:

In [ ]:
plot_coloring_problem(color_problem)

## Unsatisfiable problem

To make the problem unsatisfiable, we enforce the use of only 2 colors (we know here that the optimal number of colors is 3).


In [ ]:
solver = CpmpyColoringSolver(color_problem)
solver.init_model(nb_colors=2)
res = solver.solve()
print(solver.status_solver)
print(f"Nb of solutions found: {len(res)}")

## Minimal unsatisfiable subset
We first extract a minimal subset of conflicting constraints thanks to `cpmpy.tools.explain.mus`.

### Fine constraints
We work directly on the constraints as modelled in the cpmpy model. Here we got one constraint by edge in the graph.

In [ ]:
mus = solver.explain_unsat_fine()
mus

For coloring problems we have visualization tools available.
- we use the cpmpy wrapper to convert the constraints into edges
- we use the coloring plot utilities to highlight the corresponding edges

In [ ]:
mus_edges = solver.convert_constraints2edges(mus)
plot_coloring_problem(color_problem, highlighted_edges=mus_edges)

### Meta constraints

We can also work on higher level constraints called "meta-constraints". They are not available for all problems. It is the case for the coloring problem though.
Le us have a look to the meta-constraints defined:

In [ ]:
solver.get_soft_meta_constraints()  # Soft constraints are constraints that can be relaxed opposed to hard ones that are modelling choices or computation tricks.

Here each meta-constraint gather the constraints concerning one node: all neighbours of the given node must have a different color from its own color. Note that neighbours nodes will have their corresponding meta-constraints sharing a common fine constraint (the one corresponding to the edge between the 2 nodes).

In [ ]:
metaconstraint = solver.get_soft_meta_constraints()[2]
print(metaconstraint.name)
metaconstraint.constraints

We can query for a minimal subset of meta-constraints explaining the unsatisfiability:

In [ ]:
meta_mus = solver.explain_unsat_meta()
meta_mus

We can again visualize the subset on the graph:
- we use the cpmpy wrapper to convert the meta-constraints into nodes
- we use the coloring plot utilities to highlight the corresponding nodes

In [ ]:
meta_mus_nodes = solver.convert_metaconstraints2nodes(meta_mus)
plot_coloring_problem(color_problem, highlighted_nodes=meta_mus_nodes)

## Minimal correction subset

We can now extract a minmal subset of constraints correcting the problem. This means that without these constraints the problem can be solved. Once again there are two versions: with the fine constraints or the meta-constraints.

### Fine constraints

In [ ]:
mcs = solver.correct_unsat_fine()
mcs

Let us visualize the correction subset

In [ ]:
mcs_edges = solver.convert_constraints2edges(mcs)
plot_coloring_problem(
    color_problem,
    highlighted_edges=mcs_edges,
    highlighted_edge_style=dict(edge_color="r", width=2, style="dashed"),
)

Let us confirm that by removing the constraints we make the problem feasible.

In [ ]:
# NB: solver.model.constraints.remove(cstr) not working as expected
constraints_mcs_ids = {id(c) for c in mcs}
solver.model.constraints = [
    c for c in solver.model.constraints if id(c) not in constraints_mcs_ids
]
res = solver.solve()
print(solver.status_solver)
print(f"Nb of solutions found: {len(res)}")

In [ ]:
sol = res.get_best_solution()
plot_coloring_solution(
    sol,
    highlighted_edges=mcs_edges,
)

### Meta constraints

We first reinitialize the solver and try to solve it to be in a unsatisfiable situation.
(Else we cannot call `correct_unsat_meta()`, nor `correct_unsat_fine()`, `explain_unsat_meta()`, and `explain_unsat_fine()`.)

In [ ]:
solver = CpmpyColoringSolver(color_problem)
solver.init_model(nb_colors=2)
solver.solve()
print(solver.status_solver)

The correcting subset:

In [ ]:
meta_mcs = solver.correct_unsat_meta()
meta_mcs

Let us visualize the correction subset

In [ ]:
meta_mcs_nodes = solver.convert_metaconstraints2nodes(meta_mcs)
plot_coloring_problem(
    color_problem,
    highlighted_nodes=meta_mcs_nodes,
)

We solve without the corresponding constraints (all subconstraints covered by the meta-constraints).

In [ ]:
subconstraints_mcs_ids = set()
for meta in meta_mcs:
    subconstraints_mcs_ids.update({id(c) for c in meta.constraints})
solver.model.constraints = [
    c for c in solver.model.constraints if id(c) not in subconstraints_mcs_ids
]
res = solver.solve()
print(solver.status_solver)
print(f"Nb of solutions found: {len(res)}")

In [ ]:
sol = res.get_best_solution()
plot_coloring_solution(
    sol,
    highlighted_nodes=meta_mcs_nodes,
)